I regularly use a fitness app called Strava to track my runs and bike rides. A friend of mine told me recently that Strava has an API that allows users to access their raw data. I looked up an article on how to access it and followed the instructions (article: https://medium.com/swlh/using-python-to-connect-to-stravas-api-and-analyse-your-activities-dummies-guide-5f49727aac86). The Strava API requires authentication and uses a token system that was surprisingly complicated. Much of the code below was copied from the article. Once I was able to get the data from my most recent 200 activies into a CSV, I did a few quick data visualizations. I'm thinking that I want to use this API/data for my final project. My initial idea is to create a training log to track my weekly runs and analyze changes in fitness over time. 

In [ ]:
Establishing connection to Strava API

In [1]:
import requests
import json
import requests
from pandas.io.json import json_normalize
import csv
import time

In [5]:
# Make Strava auth API call with your 
# client_code, client_secret and code
response = requests.post(
                    url = 'https://www.strava.com/oauth/token',
                    data = {
                            'client_id': 72140,
                            'client_secret': '4e8f3558fa19f81cf9b0886f842037a36c1510b4',
                            'code': '86a4df286ceb682ba4dbcf30be1541d62af96b49',
                            'grant_type': 'authorization_code'
                            }
                )
#Save json response as a variable
strava_tokens = response.json()
# Save tokens to file
with open('strava_tokens.json', 'w') as outfile:
    json.dump(strava_tokens, outfile)
# Open JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'token_type': 'Bearer', 'expires_at': 1637530725, 'expires_in': 21600, 'refresh_token': '737b8909d96548abddd3aabbee2e610b7f478402', 'access_token': '0906682d3412c16f6ba3b4e049734441728f44c1', 'athlete': {'id': 3171594, 'username': None, 'resource_state': 2, 'firstname': 'Nick', 'lastname': 'Scott', 'bio': None, 'city': 'Brooklyn', 'state': 'New York', 'country': 'United States', 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2013-10-09T01:11:42Z', 'updated_at': '2021-09-05T15:12:29Z', 'badge_type_id': 0, 'weight': 72.5748, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/3171594/3765688/3/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/3171594/3765688/3/large.jpg', 'friend': None, 'follower': None}}


Getting first 30 activities to inspect data

In [6]:
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()
    
df = json_normalize(r)
df.to_csv('strava_activities_all_fields.csv')

C:\Users\nickx\AppData\Local\Temp/ipykernel_18760/353175613.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(r)


Automate retrieval of new tokens when old ones expire

In [7]:
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# If access_token has expired then 
# use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():
# Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': [INSERT_CLIENT_ID_HERE],
                                'client_secret': '[INSERT_CLIENT_SECRET_KEY]',
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    )
# Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
# Use new Strava tokens from now
    strava_tokens = new_strava_tokens
# Open the new JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'token_type': 'Bearer', 'expires_at': 1637530725, 'expires_in': 21600, 'refresh_token': '737b8909d96548abddd3aabbee2e610b7f478402', 'access_token': '0906682d3412c16f6ba3b4e049734441728f44c1', 'athlete': {'id': 3171594, 'username': None, 'resource_state': 2, 'firstname': 'Nick', 'lastname': 'Scott', 'bio': None, 'city': 'Brooklyn', 'state': 'New York', 'country': 'United States', 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2013-10-09T01:11:42Z', 'updated_at': '2021-09-05T15:12:29Z', 'badge_type_id': 0, 'weight': 72.5748, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/3171594/3765688/3/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/3171594/3765688/3/large.jpg', 'friend': None, 'follower': None}}


Getting 200 activities from Strava. Ready to start playing with the data!

In [8]:
## Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
## If access_token has expired then use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():
#Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': [INSERT_CLIENT_ID_HERE],
                                'client_secret': '[INSERT_CLIENT_SECRET_KEY]',
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    )
#Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
#Use new Strava tokens from now
    strava_tokens = new_strava_tokens
r = requests.get(url + '?access_token=' + access_token + '&per_page=200')
r = r.json()
    
df = json_normalize(r)
df.to_csv('strava_activities_all_fields.csv')

C:\Users\nickx\AppData\Local\Temp/ipykernel_18760/3862660779.py:26: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(r)


importing activities csv and limiting them to runs only (removing bike rides, hikes, etc.)

Another quick bar plot of the same runs as above, but displaying average heart rate (a decent approximation of effort). Starting to get some good ideas for a final project!